# Exercises

## Overview

### Questions

* What can I do to learn more about the hard sphere system?

### Objectives

* Show how to write the hard sphere simulation steps in reusable functions.
* Provide some suggested exercises for further exploration.

## Boilerplate code

In [ ]:
import hoomd
import math
import itertools
import numpy
import copy
import gsd.hoomd
import freud
import matplotlib
import IPython
%matplotlib inline
matplotlib.style.use('ggplot')

The `render` function in the next (hidden) cell will render a snapshot using **fresnel**.

<div class="alert alert-info">
    This is not intended as a full tutorial on <b>fresnel</b> - see the <a href="https://fresnel.readthedocs.io/">fresnel user documentation</a> if you would like to learn more.
</div>

In [ ]:
import fresnel

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=300, h=300)

def render(snapshot, params):
    L = snapshot.configuration.box[0]

    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.Sphere(scene,
                                       N=snapshot.particles.N,
                                       radius=params['diameter']/2)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear([0.01, 0.74, 0.26]),
                                                  roughness=0.5)
    geometry.position[:] = snapshot.particles.position[:]
    geometry.outline_width = 0.05
    box = fresnel.geometry.Box(scene, snapshot.configuration.box, box_radius=.04)
    
    scene.lights = [fresnel.light.Light(direction=(0, 0, 1), color=(0.8, 0.8, 0.8), theta=math.pi),
                    fresnel.light.Light(direction=(1, 1, 1), color=(1.1, 1.1, 1.1), theta=math.pi / 3)]
    scene.camera = fresnel.camera.orthographic(position=(0, 0, L+1),
                                               look_at=(0, 0, 0),
                                               up=(0, 1, 0),
                                               height=L+1)
    scene.background_color = (1,1,1)
    return tracer.sample(scene, samples=500)

## Workflow stages

The previous sections of this tutorial explained all of the stages of a hard sphere self-assembly simulation in individual notebooks.
Let's collect all the code from those notebooks together here in functions.
You can call these to explore simulation results as you change parameters or perform additional analysis.
First, you need a dictionary to describe the parameters of the simulation:

In [ ]:
example_params = dict(diameter=1, phi=0.57, seed=3, N_particles=500)

You will pass this dictionary into each function that needs it.

The first function makes the HPMC **Sphere** integrator based on the given parameters:

In [ ]:
def make_hard_sphere_integrator(params):
    mc = hoomd.hpmc.integrate.Sphere(seed=params['seed'])
    mc.shape['sphere'] = dict(diameter=params['diameter'])
    return mc

The next function places N particles on a cubic lattice and writes out `lattice.gsd`:

In [ ]:
def initialize(params):
    spacing = params['diameter'] * 1.1
    K = math.ceil(params['N_particles']**(1/3))
    L = K * spacing
    x = numpy.linspace(-L / 2, L / 2, K, endpoint=False)
    position = list(itertools.product(x, repeat=3))
    position = position[0:params['N_particles']]
    
    snapshot = gsd.hoomd.Snapshot()
    snapshot.particles.N = params['N_particles']
    snapshot.particles.position = position
    snapshot.particles.typeid = [0]*params['N_particles']
    snapshot.particles.types = ['sphere']
    snapshot.configuration.box = [L, L, L, 0, 0, 0]
    
    with gsd.hoomd.open(name='lattice.gsd', mode='wb') as f:
        f.append(snapshot)

The next function reads in `lattice.gsd` and runs HPMC simulations to randomize the positions of the particles and writes `random.gsd`:

In [ ]:
def randomize(params):
    cpu = hoomd.device.CPU()
    sim = hoomd.Simulation(device=cpu)
    sim.operations.integrator = make_hard_sphere_integrator(params)
    sim.create_state_from_gsd(filename='lattice.gsd')
    sim.run(10e3)
    hoomd.write.GSD.write(state=sim.state, filename='random.gsd')

The next function reads in `random.gsd` and compresses the system to the target packing fraction and writes `compressed.gsd`:

In [ ]:
def compress(params):
    cpu = hoomd.device.CPU()
    sim = hoomd.Simulation(device=cpu)
    sim.operations.integrator = make_hard_sphere_integrator(params)
    sim.create_state_from_gsd(filename='random.gsd')

    V_particle = 4/3 * math.pi * (params['diameter']/2)**3
    
    initial_box = sim.state.box
    final_box = hoomd.Box.from_box(initial_box)
    final_box.volume = sim.state.N_particles * V_particle / params['phi']
    compress = hoomd.hpmc.update.QuickCompress(trigger=hoomd.trigger.Periodic(10), seed=10, target_box = final_box)
    sim.operations.updaters.append(compress)
    
    tune = hoomd.hpmc.tune.MoveSize.scale_solver(moves=['d'],
                                                 target=0.2,
                                                 trigger=hoomd.trigger.Periodic(10),
                                                 max_translation_move=0.2)
    sim.operations.tuners.append(tune)   
    
    while not compress.complete and sim.timestep < 1e6:
        sim.run(1000)
    
    if not compress.complete:
        raise RuntimeError("Compression failed to complete")
    
    hoomd.write.GSD.write(state=sim.state, filename='compressed.gsd')

The next function reads in `compressed.gsd` and equilibrates the system over many steps, writing out `trajectory.gsd`:

In [ ]:
def equilibrate(params):
    cpu = hoomd.device.CPU()
    sim = hoomd.Simulation(device=cpu)
    sim.operations.integrator = make_hard_sphere_integrator(params)
    sim.create_state_from_gsd(filename='compressed.gsd')

    gsd = hoomd.write.GSD(filename='trajectory.gsd',
                      trigger=hoomd.trigger.Periodic(1000),
                      mode='wb')
    sim.operations.writers.append(gsd)
    
    tune = hoomd.hpmc.tune.MoveSize.scale_solver(moves=['a', 'd'],
                                                 target=0.2,
                                                 trigger=hoomd.trigger.And(
                                                     [hoomd.trigger.Periodic(100),
                                                      hoomd.trigger.Before(sim.timestep + 5000)]))
    sim.operations.tuners.append(tune)   
    sim.run(100e3)

The next function analyzes the trajectory and plots the number of particles in a solid-like environment over time:

In [ ]:
def analyze(params):
    with gsd.hoomd.open('trajectory.gsd') as traj:
        solid = freud.order.SolidLiquid(l=6, q_threshold=0.7, solid_threshold=6)
        is_solid = []
        for frame in traj:
            solid.compute(system=(frame.configuration.box, frame.particles.position),
                          neighbors=dict(mode='nearest', num_neighbors=8))
            is_solid.append(solid.num_connections > solid.solid_threshold)  

        IPython.display.display(render(traj[-1], params))
            
    fig = matplotlib.figure.Figure(figsize=(10, 6.18))
    ax = fig.add_subplot()
    num_solid = [numpy.sum(a) for a in is_solid]
    ax.plot(num_solid)
    ax.set_xlabel('frame')
    ax.set_ylabel('number of particles in a solid environment')
    ax.set_ylim([0, params['N_particles']])
    IPython.display.display(fig)

The final function combines the `initialize`, `randomize`, `compress`, and `equilibrate` steps into one function.

In [ ]:
def run_simulation(params):
    initialize(params)
    randomize(params)
    compress(params)
    equilibrate(params)

## Run the simulation

Call the `run_simulation` function defined above to run the simulation with the given parameters:

In [ ]:
sim_params = dict(diameter=1, phi=0.57, seed=1, N_particles=500)
run_simulation(sim_params)

Analyze the results of the simulation:

In [ ]:
analyze(sim_params)

## Exercises

Try changing the simulation parameters above and rerunning the simulation.

* What happens when you change the random number seed?
* Do all seeds lead to the same final structure?
* Can you find a seed where all ~500 particles in the box end in a solid-like environment?
* How does the system behave at lower packing fractions (phi)?
* At what packing fraction does the system start to crystallize?
* How does the behavior of the system change with the number of particles?

[Previous section](07-Analyzing-Trajectories.ipynb).